In [12]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.torch_core import *
from fastai.callback import *
from fastai.basic_train import *
import torchvision.models as tmodels

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from scipy.misc import imsave,imread

import os
import sys

sys.path.insert(0, '/home/santhosr/Documents/Birad/FastAI/RaceClassification/withLargerDataset/' )

from utils import *

inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

df1 = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')
df1.drop(['PresIntentType','DBT'],inplace = True,axis=1)


df2 = pd.read_csv('/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv')
df2.Medview_Race = 'White'
truth = pd.concat([df1,df2],sort=True)

In [2]:
truth.head()

,Age,BMI,Density_Overall,DummyID,Medview_Race,ScreenDate
0,67.8,27.1,2.0,2508626,White,3/23/2011
1,58.8,25.7,2.0,2505738,White,6/2/2011
2,56.4,28.1,2.0,2547962,White,5/24/2011
3,51.7,31.8,2.0,2575490,White,5/6/2011
4,63.7,27.5,2.0,2591080,Other,5/5/2011


In [3]:
truth.Medview_Race.value_counts()

African American         10369
White                     5916
Other                      214
Asian                      205
Unknown                     67
Hispanic Latino/White       51
Hispanic Latino/Black       22
East Indian                 16
Pacific Island               6
American Indian              5
Name: Medview_Race, dtype: int64

In [4]:
truth=truth.drop_duplicates()

In [5]:
df = createDataset2(seed=123,n=5000)
# df = upSampleDf(df)
df= df.sample(frac=1)

8405
30753
13438
816
53412
64527
total number of patients 16108


/home/santhosr/Documents/Birad/FastAI/RaceClassification/withLargerDataset/utils.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainTemp['train'] = False
/home/santhosr/Documents/Birad/FastAI/RaceClassification/withLargerDataset/utils.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  validTemp['train'] = True


In [6]:
def getRaceLabel(x,binary=False):
    ID = x.split("/")[-1].split("_")[0]
    label = truth[truth.DummyID == int(ID)]['Medview_Race'].values[0]

    if label == 'African American':
        return 0
    elif label == "White":
        return 1
    else:
        return 2
    
df['Race'] = df.filename.apply(lambda x : getRaceLabel(x))

In [13]:
t = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')

t.head()

,BMI,Medview_Race,ScreenDate,Age,DummyID,PresIntentType,DBT,Density_Overall
0,27.1,White,3/23/2011,67.8,2508626,FOR PROCESSING,NaN,2
1,25.7,White,6/2/2011,58.8,2505738,FOR PROCESSING,NaN,2
2,28.1,White,5/24/2011,56.4,2547962,FOR PROCESSING,NaN,2
3,31.8,White,5/6/2011,51.7,2575490,FOR PROCESSING,NaN,2
4,27.5,Other,5/5/2011,63.7,2591080,FOR PROCESSING\,No Tomo images,2


In [24]:
df1.head()

,BMI,Medview_Race,ScreenDate,Age,DummyID,Density_Overall
0,27.1,White,3/23/2011,67.8,2508626,2
1,25.7,White,6/2/2011,58.8,2505738,2
2,28.1,White,5/24/2011,56.4,2547962,2
3,31.8,White,5/6/2011,51.7,2575490,2
4,27.5,Other,5/5/2011,63.7,2591080,2


In [8]:
# df.to_csv("DataFile5.csv",index=False,index_label=False)

# df = pd.read_csv("DataFile.csv")



class ModelTrackerCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model when monitored quantity is best."
    def __init__(self, learn:Learner, path:str='/home/santhosr/Documents/Birad/ProcessedData/models',id:int=None,monitor:str='val_loss', mode:str='auto',modelName:str='resnet50'):
        super().__init__(learn, monitor=monitor, mode=mode)
        
        self.bestAcc = 0.0001
        self.folderPath = path
        self.id = id
        self.modelName = modelName
        super().__post_init__()

    def on_epoch_end(self, epoch, **kwargs:Any)->None:
        "Compare the value monitored to its best score and maybe save the model."

        acc = float(self.learn.recorder.metrics[epoch-1][0])
        val_loss = self.learn.recorder.val_losses[epoch-1]

        if acc>self.bestAcc:
            self.bestAcc = acc
            if self.id==None:
                fileName = 'model_'+self.modelName+'_acc'+str(int(acc*1000))+"_loss"+str(int(val_loss*1000))
            else:
                fileName = 'model_'+self.modelName+'_id' + str(self.id) + '_acc' + str(int(acc*1000)) + "_loss" + str(int(val_loss*1000))
            fileName = os.path.join(self.folderPath, fileName)
            self.learn.save(fileName)


def getRaceLabel(x,binary=False):
    ID = x.split("/")[-1].split("_")[0]
    label = truth[truth.DummyID == int(ID)]['Medview_Race'].values[0]

    if label == 'African American':
        return 0
    elif label == "White":
        return 1
    else:
        return 2
    

In [9]:
#Creates the FastAI Dataset
data = ImageItemList.from_df(df=df,path='/home/santhosr/Documents/Birad/ProcessedData/', cols='filename').split_from_df(col='train').label_from_func(getRaceLabel).transform(get_transforms(),size=256).databunch(bs=50).normalize()
print("Dataset created")

Dataset created


In [10]:
#Creates the model architecture 
learn = create_cnn(data, tmodels.resnet50, metrics=accuracy,pretrained=True)

In [11]:
# learn.load('/home/santhosr/Documents/Birad/ProcessedData/models/model_resnet50_acc668_loss600')

# best_model_cb = partial(ModelTrackerCallback,id=6, modelName = "resnet50")
# learn.callback_fns.append(best_model_cb)

#Unfreezes the entire model
learn.unfreeze()

#Uses a fixed learning rate of 1e-5
learn.fit(30,1e-5)

learn.unfreeze()
learn.fit_one_cycle(10,slice(1e-5))
learn.fit_one_cycle(10,slice(1e-5))
learn.fit(5,slice(1e-4))

learn.freeze()
learn.fit(3,1e-2)

learn.unfreeze()
learn.fit_one_cycle(5,1e-3)
learn.fit(5,1e-4)

epoch,train_loss,valid_loss,accuracy


OSError: Traceback (most recent call last):
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py", line 544, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py", line 94, in __getitem__
    if isinstance(idxs, numbers.Integral): return self.get(idxs)
  File "/home/santhosr/.local/lib/python3.6/site-packages/fastai/vision/data.py", line 266, in get
    res = self.open(fn)
  File "/home/santhosr/.local/lib/python3.6/site-packages/fastai/vision/data.py", line 262, in open
    return open_ima

In [9]:
from scipy.misc import imread

In [12]:
a=imread('/home/santhosr/Documents/Birad/ProcessedData/FullRes/2/4077998_FOR-PROCESSING_L_MLO_1.jpg')

/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


FileNotFoundError: [Errno 2] No such file or directory: '/home/santhosr/Documents/Birad/ProcessedData/FullRes/2/4077998_FOR-PROCESSING_L_MLO_1.jpg'